In [11]:
import pandas as pd 
import numpy as np
import html

In [15]:
questions = pd.read_csv("Dataset/Questions.csv",encoding='latin-1')
answers = pd.read_csv("Dataset/Answers.csv",encoding='latin-1')
tags = pd.read_csv("Dataset/Tags.csv",encoding='latin-1')
tag_question = tags.groupby('Id').agg(list).merge(questions,how='inner',on = "Id")
df = tag_question.merge(answers,how = "inner",left_on = "Id", right_on = "ParentId")
df.columns = df.columns.str.replace("_x","_question").str.replace("_y","_answer")
df = df[['Id','Score_question','Title','Body_question',"Score_answer","Body_answer"]]

In [16]:
best_answers = df.loc[df.groupby("Id")['Score_answer'].idxmax()]

In [17]:
best_answers['total_score'] = best_answers['Score_answer'] + best_answers['Score_question']

In [18]:
best = best_answers.sort_values('total_score',ascending=False)[:50000]

In [19]:
best

,Id,Score_question,Title,Body_question,Score_answer,Body_answer,total_score
5845,231767,5524,"What does the ""yield"" keyword do?",<p>What is the use of the <code>yield</code> k...,8384,"<p>To understand what <code>yield</code> does,...",13908
2233,100003,3219,What is a metaclass in Python?,<p>What are metaclasses? What do you use them ...,4510,<h1>Classes as objects</h1>\n\n<p>Before under...,7729
10416,394809,2655,Does Python have a ternary conditional operator?,<p>If Python does not have a ternary condition...,3110,"<p>Yes, it was <a href=""https://mail.python.or...",5765
20287,739654,1920,How to make a chain of function decorators in ...,<p>How can I make two decorators in Python tha...,3332,"<p>If you are not into long explanations, see ...",5252
1902,82831,2729,How do I check whether a file exists using Pyt...,"<p>How do I check whether a file exists, witho...",2356,"<p>You can also use <a href=""https://docs.pyth...",5085
...,...,...,...,...,...,...,...
289005,11845238,4,Utilizing %r within raw_input in python,<p>Is it feasible to utilize %r within raw_inp...,7,<p>Your line should read </p>\n\n<pre><code>ra...,11
290775,11918735,5,Error trying to login to webpage using seleniu...,<p>I get an element isn't visible error:</p>\n...,6,<p>Selenium interacts with the web browser in ...,11
160698,5996616,3,Efficient way to delete lines from a file in p...,<p>My program is keeping a log for the user. ...,8,<p>The general method to achieve this for logf...,11
87144,3086514,4,Python memory usage: Which of my objects is ho...,<p>The program I've written stores a large amo...,7,"<p>The floats do take up 16 bytes apiece, and ...",11


In [21]:
best['Body_question'] = (best['Body_question']
    .apply(html.unescape)
    .str.replace(r'<[a-zA-Z/][^>]*>', '', regex=True)  #remove HTML tags
    .str.replace('\r','')
    .str.replace('’',"'"))

In [22]:
best['Body_answer'] = (best['Body_answer']
    .apply(html.unescape)
    .str.replace(r'<[a-zA-Z/][^>]*>', '', regex=True)  #remove HTML tags
    .str.replace('\r','')
    .str.replace('’',"'"))

In [23]:
best['Title'] = (best['Title']
    .str.replace(r'  +', ' ', regex=True)              #collapse multiple spaces
    .str.replace('\r','')
    .str.replace('’',"'"))

In [25]:
best['quesiton'] = best['Title'] + " " + best['Body_question']

In [27]:
best = best[['quesiton','Body_answer']]

In [29]:
best.to_csv("bestsubset.csv")